In [ ]:
# Author : Raude Killian
# Last modified 16.07.205
# Last Check by Qiaoyue 15.07.2025

In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import warnings

MEG_CALIBRATION_FILE = rf"meg-wiki-datasets-main\calibration\sss_cal_20250327.dat"
MEG_CROSSTALK_FILE = rf"meg-wiki-datasets-main\cross-talk\ct_sparse.fif"

PARTICIPANTS_FILE = rf"L:\Common\Users\Qiaoyue\MEG_project\Data\participants.csv"

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
mne.set_log_level('ERROR')

participants = pd.read_csv(PARTICIPANTS_FILE)  
participants = participants['subID'].astype(str).tolist()
sessions = ["encoding", "recall"]

plots = FALSE

print(f"✅ {len(participants)} participants list : {participants}")

In [ ]:
# For tests purposes
#participants = ["F103"]
#sessions = ["encoding"]

In [ ]:
# Extract the device to head transform for each file and average across participants

from scipy.spatial.transform import Rotation as R
from mne.transforms import Transform
rotations = []
translations = []

for subID in participants:

    plots_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\plots\{subID}"
    os.makedirs(plots_folder, exist_ok=True)

    #intermediates_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\meg_intermediates\{subID}"
    intermediates_folder = rf"C:\Users\killg\Desktop\meg_intermediates\{subID}"
    os.makedirs(intermediates_folder, exist_ok=True)

    for session in sessions:
        part = "ERROR"
        if(session == "encoding"): 
            part = "part1"
        elif(session == "recall"):
            part = "part2"

        # Declare filenames 
        raw_meg_filename = rf"C:\Users\killg\Desktop\Data\{subID}\MEG\{part}.fif"
        #raw_meg_filename = rf"L:\Common\Users\Qiaoyue\MEG_project\Data\{subID}\MEG\{part}.fif"

        if not map(os.path.exists, raw_meg_filename):
            print(f"❌ Missing file for participant {subID}. Skipping...")
            continue
        print(f"Getting head position from: {subID}_{session}_session ...")

        raw_meg_file = mne.io.read_raw_fif(raw_meg_filename, preload=True, verbose=False)
        dev_head_t = raw_meg_file.info["dev_head_t"]

        rot_matrix = dev_head_t['trans'][:3, :3]
        trans_vector = dev_head_t['trans'][:3, 3]

        # Convert rotation to quaternion
        quat = R.from_matrix(rot_matrix).as_quat()
        rotations.append(quat)
        translations.append(trans_vector)

rotations = np.array(rotations)
translations = np.array(translations)

avg_quat = rotations.mean(axis=0)
avg_quat /= np.linalg.norm(avg_quat)  

avg_rot_matrix = R.from_quat(avg_quat).as_matrix()
avg_translation = translations.mean(axis=0)

avg_dev_head_t = Transform('meg', 'head', np.eye(4))
avg_dev_head_t['trans'][:3, :3] = avg_rot_matrix
avg_dev_head_t['trans'][:3, 3] = avg_translation

print(f"✅ Average dev_head_t_{session} ready for maxwell_filter:")
print(avg_dev_head_t)

In [ ]:
# Filtering 
for subID in participants:
    
    plots_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\plots\{subID}"
    os.makedirs(plots_folder, exist_ok=True)
    
    #intermediates_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\meg_intermediates\{subID}"
    intermediates_folder = rf"C:\Users\killg\Desktop\meg_intermediates\{subID}"
    os.makedirs(intermediates_folder, exist_ok=True)

    for session in sessions:

        part = "ERROR"
        if(session == "encoding"): 
            part = "part1"
        elif(session == "recall"):
            part = "part2"

        # Declare filenames 
        raw_meg_filename = rf"C:\Users\killg\Desktop\Data\{subID}\MEG\{part}.fif"
        #raw_meg_filename = rf"L:\Common\Users\Qiaoyue\MEG_project\Data\{subID}\MEG\{part}.fif"

        if not map(os.path.exists, raw_meg_filename):
            print(f"❌ Missing file for participant {subID}. Skipping...")
            continue
        print(f"Preprocessing: {subID}_{session}_session")

        raw_meg_file = mne.io.read_raw_fif(raw_meg_filename, preload=True, verbose=False)

        # At time of recording, MEG1041 was dead, remove from the file so it does not affect the filtering.
        raw_meg_file.info['bads'] = ['MEG1041'] 
        

        # Only drop for better PSD plots, keep bad and interpolate later for the rest of the analysis
        #raw_meg_file.drop_channels(raw_meg_file.info['bads'])


        # Compute and save raw PSD plots
        if(plots):
            n_fft = 2000
            PSD_data = raw_meg_file.compute_psd(method="welch", fmin=1, fmax=60, picks="meg", n_fft=n_fft, n_overlap=int(n_fft/2), verbose = False)
            fig = PSD_data.plot(show=False) 
            fig.suptitle(f"Power Spectral Density: {subID}_{session} Raw", fontsize=16)
            fig.savefig(f"{plots_folder}/{subID}_{session}_Raw.png")
            plt.close(fig)
        

        raw_meg_file.fix_mag_coil_types()
 
        # Check which channels are noisy, do not include them in the maxwell calculations so they do not propagate their noise
        auto_noisy_chs, auto_flat_chs, auto_scores = mne.preprocessing.find_bad_channels_maxwell(
        raw_meg_file, 
        cross_talk=MEG_CROSSTALK_FILE, 
        calibration=MEG_CALIBRATION_FILE,
        return_scores=True, 
        verbose=False)

        if(len(auto_noisy_chs) > 0):
            print('Noisy channels =', auto_noisy_chs)
        if(len(auto_flat_chs) > 0):
            print('Flat channels =', auto_flat_chs)

        raw_meg_file.info['bads'].extend(auto_noisy_chs + auto_flat_chs)


        # Apply maxwell filtering
        sss_meg_file = mne.preprocessing.maxwell_filter(
        raw_meg_file,
        cross_talk=MEG_CROSSTALK_FILE, 
        calibration=MEG_CALIBRATION_FILE,
        destination = avg_dev_head_t,
        verbose=False)


        # Reapply previous bad channels because maxwell filter can forget these
        sss_meg_file.info['bads'] = raw_meg_file.info['bads']

        sss_meg_file.filter(0.1,40., verbose=False, picks = ["meg"])
        sss_meg_file.save(f"{intermediates_folder}/{subID}_{session}_sss.fif", overwrite=True, verbose = False)


        # Compute and save sss PSD plots
        if(plots):
            PSD_data = sss_meg_file.compute_psd(method="welch", fmin=1, fmax=60, picks="meg", n_fft=n_fft, n_overlap=int(n_fft/2), verbose = False)
            fig = PSD_data.plot(show=False) 
            fig.suptitle(f"Power Spectral Density: {subID}_{session} SSS", fontsize=16)
            fig.savefig(f"{plots_folder}/{subID}_{session}_SSS.png")
            plt.close(fig)

    print(f"✅ Participant: {subID} sucessfully completed  \n")